In [49]:
# Leitura Dataframes e resultados
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Enviar reqs para o bot
import requests

#Acessar print do sistema
import sys

#Libs para gerar o tempo
import time
import calendar

# Classificadores
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

#Grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#Feature Selection
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Example
from sklearn import datasets

In [50]:
#Dados de configuração do bot
TELEGRAM_TOKEN = "TOKEN_HERE"
  # TOKEN example 5298243636:AAGFB9MK3NyrJd9FstfhgVM0Pq95SCEZqfg
CHAT_ID = "CHAT_ID"
# CHAT EXAMPLE -761539976
EXPERIMENTO_ID = "ID" #Id pra diferenciar cada experimento

#Configuração do Experimento
K_FOLD = 2
MODELO = AdaBoostClassifier()
SEED = 42 #Resposta final da vida, do universo e tudo

#base de dados
# database_path = "/home/ubuntu/base_dados/com_undersampling/baseCompleta-1617664035.csv"
output_feature = "target"
results_path = "/home/thomas/Documentos/mestrado/results"

'''
Array de parametros para rodar no gridSearch
parametros do modelo sao iniciados com "sfs__estimator__"
parametros do sfs sao iniciados com "sfs__"
'''
param_grid = [{
    # 'sfs__forward': [True, False],
    # 'sfs__floating': [True, False],
    # 'sfs__estimator__n_estimators': [25,50,100],
    'learning_rate': [0.36, 1., 1.5],
}]

total_comb = K_FOLD
for key in param_grid[0].keys():
  total_comb = total_comb * len(param_grid[0][key])

timestamp = calendar.timegm(time.gmtime())

In [51]:
def print_bot(bot_message):
    '''
    Descrição:
        Envia a mensagem desejada para o chat desejado atraves do bot do telegram
    Entrada:
        bot_message
            string com a mensagem
    Var_Config:
        TELEGRAM_TOKEN:
            Token do bot do telegram
        CHAT_ID:
            id do chat para onde a mensagem sera enviada
        EXPERIMENTO_ID:
            Id de identificacao para saber de qual experimento será a mensagem
    '''
    
    bot_message = EXPERIMENTO_ID+"->"+bot_message
    bot_token = TELEGRAM_TOKEN
    bot_chatID = CHAT_ID
    # url BASE https://api.telegram.org/bot<BOT_TOKEN/sendMessage?chat_id=<CHAT_ID>&text=<MESSAGE>
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [52]:
'''
Com esse bloco todo print feito no projeto será encaminhado para o bot e salvo em um arquivo de log
O nome do arquivo de log é feito com base no timestamp de quando o experimento foi iniciado
O atributo 'bot_print' serve para impedir de ser enviado para o bot, em casos onde as mensagem serao muitos grandes
No experimento quando e printado todos os resultados do grid search, nao e enviado para o bot

No ultimo bloco de codigo e necessario o comando 'sys.stdout = orig_stdout' para desativar a classe e o arquivo 
de log ser salvo
'''

nome_arquivo = "relatorio-"+EXPERIMENTO_ID+"-"+str(timestamp)+".log"
path_log = results_path + "/"+nome_arquivo

class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout
        self.log = open(path_log, "a")
        self.contador = 0

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        if (message not in "\n"):
          if (message.startswith('[CV]')):
            self.contador = self.contador + 1
            # if ((self.contador%2) == 0 and self.contador != 0):
            info = (f"{self.contador}/{total_comb} concluídos✅")
            message = message + "\n" + info
          print_bot(message)

    def flush(self):
        #this flush method is needed for python 3 compatibility.
        #this handles the flush command by doing nothing.
        #you might want to specify some extra behavior here.
        pass  

orig_stdout = sys.stdout
sys.stdout = Logger()

In [53]:
#Leitura e divisao da base de dados
# df = pd.read_csv(database_path, sep=";")
df = datasets.load_iris()
# print(f'Initial shape: {df.shape}')

# X = df.drop(output_feature, axis=1)
X = df.data
y = df[output_feature]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

In [54]:
'''
Iniciando as libs

O SFS sera executado com n_jobs=-1 para aproveitar todo o processamento da maquina
O subconjuhnto sera avaliado pela accuracy

O gridSearch sera executado com n_jobs=1 para ser possivel observar sua evolucao, alem de que o SFS ja estara co n_jobs=-1,
entao nao sera necessario manter o -1 nele tambem
a selecao da melhor configuracao sera feita atraves da accuracy

No fim sera possivel adquirir o relatorio de feature selection apenas da melhor configuracao, para analises posteriores
a acuracia de todos as combinacoes tambem sera salva no arquivo de log para analise posterior
'''

sfs = SFS(
  estimator=MODELO, 
  k_features=(1,X.shape[1]), 
  scoring='accuracy',
  cv=K_FOLD,
  n_jobs=-1
)

pipe = Pipeline([
  ('sfs', sfs), 
  ('modelo', MODELO)])

gs = GridSearchCV(
  estimator=MODELO, 
  param_grid=param_grid, 
  scoring='accuracy', 
  n_jobs=-1, 
  cv=K_FOLD,
  refit=True,
  verbose=4
)

In [55]:
#Execucao do experimento, caso algum erro ocorra durante o experimento, a val ira avisar

try:
    print('Iniciando grid search 💻')
    gs.fit(X_train,y_train)
    print('Finalizado 🤩🥳🎉')
    
    #melhores resultados
    # print(f'🔎Best features: {gs.best_estimator_.steps[0][1].k_feature_names_}')
    print(f'⚙️Best params: {gs.best_params_}')
    print(f'📊Acc: {gs.best_score_}')
    
    # #salvando o dataframe com o relatorio das features do melhor
    # rel_sfs = gs.best_estimator_.steps[0][1]
    # results_df = pd.DataFrame.from_dict(rel_sfs.get_metric_dict()).T
    # results_df.to_csv(f'{results_path}/sfs_best-{EXPERIMENTO_ID}-{str(timestamp)}.csv', index=False)
    
    #salvando dataframe do gridSearch
    gs_df = pd.DataFrame(gs.cv_results_)
    gs_df.to_csv(f'{results_path}/grid_search-{EXPERIMENTO_ID}-{str(timestamp)}.csv', index=False)

except Exception as e:
    print(e)
    print(f'Infelizmente, ocorreu um erro 😔')
print(f'Finalizei o meu trabalho, até a próxima 😉')

Iniciando grid search 💻
Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV 2/2] END .................learning_rate=1.5;, score=0.904 total time=   0.2s
[CV 2/2] END .................learning_rate=1.0;, score=0.923 total time=   0.2s
[CV 1/2] END .................learning_rate=1.0;, score=0.887 total time=   0.2s
[CV 1/2] END .................learning_rate=1.5;, score=0.887 total time=   0.2s
[CV 1/2] END ................learning_rate=0.36;, score=0.887 total time=   0.1s
[CV 2/2] END ................learning_rate=0.36;, score=0.904 total time=   0.1s
Finalizado 🤩🥳🎉
⚙️Best params: {'learning_rate': 1.0}
📊Acc: 0.9049346879535559
Finalizei o meu trabalho, até a próxima 😉


In [56]:
#Setando a saida para o padrao e salvando arquivo de log
sys.stdout = orig_stdout